# EXPORTED THE MONGODB COLLECTION
I exported the `iwp` collection in the `etl_db` to the file `mongodb_export_iwp.json` using
```mongoexport --db etl_db --collection iwp --out mongodb_export_iwp.json```


# IMPORT THE MONGODB COLLECTION
The JSON can be imported to the `iwp` collection in the `etl_db` database from file `mongodb_export_iwp.json` using
```mongoimport --db etl_db --collection iwp --file mongodb_export_iwp.json```


# DOCUMENT FORMAT
The documents in the etl_db collection iwp have the following formats:
1. United States - Address is parsed
<br>

```{'_id': ObjectId('5c6a8db6aaa45be6f85938c7'),
 'incident_title': "BJ's Restaurant & Brewhouse, Red Bug Lake Road, Oviedo, FL, USA",
 'incident_date': 'Dec 30 2018 10:51pm',
 'incident_url': 'https://iwaspoisoned.com/incident/bjs-restaurant-brewhouse-red-bug-lake-road-oviedo-fl-usa-155865#emailscroll',
 'incident_report_type': 'Food Poisoning',
 'incident_symptoms': ['Diarrhea', 'Nausea', 'Vomiting'],
 'incident_misc': '',
 'incident_address_info':
     {'incident_address': '8285 Red Bug Lake Road, Oviedo, 32765 Florida, United States',
      'incident_address_standard': '8285 Red Bug Lake Road, Oviedo, Florida 32765, United States',
      'incident_address_street': '8285 Red Bug Lake Road',
      'incident_address_street2': '',
      'incident_address_street3': '',
      'incident_address_city': 'Oviedo',
      'incident_address_state': 'Florida',
      'incident_address_zipcode': '32765',
      'incident_address_country': 'United States'},
 'incident_description': 'Ate there Friday evening, got sick about 8 hours later'}```

<br>


2. Other Countries - Address is not parsed
(Should only USA entries in the database due to filtering, but just in case...)
<br>

```{'_id': ObjectId('5c6a8db6aaa45be6f85938c7'),
 'incident_title': "BJ's Restaurant & Brewhouse, Red Bug Lake Road, Oviedo, FL, USA",
 'incident_date': 'Dec 30 2018 10:51pm',
 'incident_url': 'https://iwaspoisoned.com/incident/bjs-restaurant-brewhouse-red-bug-lake-road-oviedo-fl-usa-155865#emailscroll',
 'incident_report_type': 'Food Poisoning',
 'incident_symptoms': ['Diarrhea', 'Nausea', 'Vomiting'],
 'incident_misc': '',
 'incident_address_info':
     {'incident_address': '8285 Red Bug Lake Road, Oviedo, 32765 Florida, United States'},
 'incident_description': 'Ate there Friday evening, got sick about 8 hours later'}```

<br>


In [1]:
# Extract Data from iwaspoisoned.com website using web scraping.
# Then populate the information in a MongoDB
# (to facilitate teaming, export the MongoDB to a JSON file)

In [2]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine
from pprint import pprint

from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo

import time

# Support export to and import from JSON file
import json

In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
# Define database and collection
db = client.etl_db

In [5]:
# Setup the splinter Browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
# FUNCTION: get_incident_detail()
# This function accepts a url that points to a single incident detail page
# and returns a dictionary with info from that page.
#
# Note: This function performs special parsing of addresses in the United States.
# Addresses in the US will be parsed down to individual components
# (street, street2, street3, city, state, zipcode, country)
# In addition, these parsed components are then recombined
# to form the full address in "standard" format
# (i.e., Zipcode after the state instead of before state)
#
# Addresses for other countries are provided only as an address string
#
# Arguments:
#    incident_detail_url: URL of the incident detail page
#
# Returns:
#    retval: A dictionary containing the incident detail page info

def get_incident_detail(a_url):

    # URL of page to be scraped
    # url_incident = 'https://iwaspoisoned.com/incident/chick-fil-a-north-fairfield-road-beavercreek-oh-usa-168576#emailscroll'
    # url_incident = 'https://iwaspoisoned.com/incident/subway-terminal-3-silver-dart-drive-toronto-on-canada-168642#emailscroll'
    if len(a_url) == 0:
        return None
    
    url_incident = a_url
    
    # Retrieve page with the requests module
    response = requests.get(url_incident)

    # Create BeautifulSoup object; parse with 'lxml'
    soup = BeautifulSoup(response.text, 'lxml')

    # Examine the results, then determine element that contains sought info
    # results are returned as an iterable list
    results = soup.find_all('div', class_='single-incident')

    for r in results:
        # Incident detail page - title
        # incident_detail_title = r.find('h1', class_='h1 post-title').text.strip()

        # Address
        try:
            addr_info = r.find('span', class_='pl-1 py-0 text-muted').text.strip()
            incident_address = ' '.join(addr_info.split())
            
        except AttributeError:
            addr_info = ""
            incident_address = ""

        # Ok, we now have an address of the form:
        # 2360 North Fairfield Road, Beavercreek, 45431 Ohio, United States
        # But, would be nice to be able to break this up into
        # individual components to facilitate address matching,
        # Especially with the non-standard location of the zipcode
        if "United States" in incident_address:
            # Create a list of address items
            ai_list = incident_address.split(',')

            # Some items are mandatory and are at the end of the list of length = N
            # N-1: Country e.g. "United States"
            # N-2: Zipcode and State e.g. "45431 Ohio"
            # N-3: City
            # Other entries 0 to N-4: Street/Apt/etc.

            # Get the count of how many components are in the address
            ai_size = len( ai_list )
            
            # Country
            try:
                incident_address_country = ai_list[ai_size-1].strip()
            except IndexError:
                incident_address_country = ""

            # Split the next entry to get state and zipcode
            try:
                zs_info = ai_list[ai_size-2].strip()
                zs_delim = zs_info.find(' ')
                # print(f"zs_delim: {zs_delim}, zs_info: {zs_info}")
                incident_address_zipcode = zs_info[:zs_delim].strip()
                incident_address_state = zs_info[zs_delim:].strip()

            except IndexError:
                incident_address_zipcode = ""
                incident_address_state = ""
                
            # City
            try:
                incident_address_city = ai_list[ai_size-3].strip()
                
            except IndexError:
                incident_address_city = ""

            # Process up to 3 "street" type entries
            incident_address_street = ""
            incident_address_street2 = ""
            incident_address_street3 = ""

            # print(f"ai_size: {ai_size}")
            # First street address item
            if ai_size >= 4:
                try:
                    incident_address_street = ai_list[0].strip()
                except:
                    incident_address_street = ""

            # Second street address item
            if ai_size >= 5:
                try:
                    incident_address_street2 = ai_list[1].strip()

                except:
                    incident_address_street2 = ""

            # Third street address item
            if ai_size >= 6:
                try:
                    incident_address_street3 = ai_list[i].strip()
                
                except:
                    incident_address_street3 = ""

            # Rebuild the address - with standard state then zipcode formating
            incident_address_standard = incident_address_street
            
            if len(incident_address_street2) > 0:
                incident_address_standard += ", " + incident_address_street2
            if len(incident_address_street3) > 0:
                incident_address_standard += ", " + incident_address_street3
            if len(incident_address_city) > 0:
                incident_address_standard += ", " + incident_address_city
            if len(incident_address_state) > 0:
                incident_address_standard += ", " + incident_address_state
            if len(incident_address_zipcode) > 0:
                incident_address_standard += " " + incident_address_zipcode
            if len(incident_address_country) > 0:
                incident_address_standard += ", " + incident_address_country


            #print(f">>> Incident Detail - Address: {incident_address}")
            #print(f">>> Incident Detail - Address - Standard: {incident_address_standard}")
            #print(f">>> Incident Detail - Address - Street: {incident_address_street}")
            #print(f">>> Incident Detail - Address - Street2: {incident_address_street2}")
            #print(f">>> Incident Detail - Address - Street3: {incident_address_street3}")
            #print(f">>> Incident Detail - Address - City: {incident_address_city}")
            #print(f">>> Incident Detail - Address - State: {incident_address_state}")
            #print(f">>> Incident Detail - Address - Zipcode: {incident_address_zipcode}")
            #print(f">>> Incident Detail - Address - Country: {incident_address_country}")
            #print("-"*40)

            # Place all this good info into a dictionary
            detail_post_item = {
                'incident_address': incident_address,
                'incident_address_standard': incident_address_standard,
                'incident_address_street': incident_address_street,
                'incident_address_street2': incident_address_street2,
                'incident_address_street3': incident_address_street3,
                'incident_address_city': incident_address_city,
                'incident_address_state': incident_address_state,
                'incident_address_zipcode': incident_address_zipcode,
                'incident_address_country': incident_address_country
            }

        else:
            # Ok, for other countries, don't try to parse the incident_address
            # print(f">>> Incident Detail - Address: {incident_address}")
            # print("-"*40)

            # Place all this good info into a dictionary
            detail_post_item = {
                'incident_address': incident_address,
            }

        # pprint(detail_post_item)
        
        return detail_post_item
    


In [7]:
# FUNCTION: parse_one_incident()
# This function accepts a Beautiful Soup object that contains a single incident
# and returns a dictionary with info for that incident.
# This includes a call to the get_incident_detail() function,
# which gets needed information from the detail page for this incident
#
# Arguments:
#    a_bsobj: A Beautiful Soup object containing a single incident
#
# Returns:
#    retval: A dictionary containing the incident detail page info

def parse_one_incident(a_bsobj):
    
    # Create BeautifulSoup object; parse with 'lxml'
    r = a_bsobj

    # Get the primary incident report info from the main box
    main_box = r.find('div', class_='report-first-box')
    
    # Date the incident occurred
    try:
        incident_date = main_box.find('p', class_ = 'report-date').text.strip()
        
    except AttributeError:
        incident_date = ""
        

    # Title of the incident
    try:
        incident_title = main_box.find('a')['title']

    except AttributeError:
        incident_title = ""
    
    # Remove the tag phrase from the title if it's present
    if "- Got Food Poisoning? Report it now" in incident_title:
        i_delim = incident_title.find("- Got Food Poisoning? Report it now")
        incident_title = incident_title[:i_delim].strip()

    # URL of the per-incident details
    try:
        incident_url = main_box.find('a')['href'].strip()

    except AttributeError:
        incident_url = ""

    # Get the Symptoms
    report_tags = main_box.find_all('p', class_ = 'report-tag')

    # Parse each report tag into its proper field
    incident_symptoms = ""
    incident_report_type = ""
    incident_misc = ""

    for rt in report_tags:
        # Get the text in this tag
        rt_info = rt.text.strip()

        # Symptoms
        if "Symptoms:" in rt_info:
            incident_symptoms = [ s.replace(',','') for s in rt_info[len("Symptoms: "):].split() ]

        # Report Type
        elif "Report Type:" in rt_info:
            incident_report_type = rt_info[len("Report Type: "):]

        # Ok... no idea what this report tag contains
        else:
            incident_misc = rt_info

    #pprint(main_box)
    #print(f">>> Incident Date: {incident_date}")
    #print(f">>> Incident Title: {incident_title}")
    #print(f">>> Incident URL: {incident_url}")
    #print(f">>> Incident Report Type: {incident_report_type}")
    #print(f">>> Incident Symptoms: {incident_symptoms}")
    #print(f">>> Incident Misc Info: {incident_misc}")
    #print("-"*40)

    # Get the full description of the incident
    # Assume this couple be populated in multiple paragraphs
    desc_box = r.find('div', class_='report-second-box')
    desc_list = desc_box.find_all('p')
    incident_description = ""
    for d in desc_list:
        incident_description += d.text.strip()

    #pprint(descbox)
    #print(f">>> Description: {incident_description}")
    #print("-"*40)

    # Go to the detail page to get the one piece of info we
    # need that's not on the main page - the address!
    incident_address_info = get_incident_detail(incident_url)

    # Place all this good info into a dictionary
    post_item = {
        'incident_title': incident_title,
        'incident_date': incident_date,
        'incident_url': incident_url,
        'incident_report_type': incident_report_type,
        'incident_symptoms': incident_symptoms,
        'incident_misc': incident_misc,
        'incident_address_info': incident_address_info,
        'incident_description': incident_description
    }
    #pprint(post_item)

    return post_item


In [8]:
# FUNCTION: parse_incident_page()
# This function accepts an HTML string from an
# IWP website page that contains multiple incidents.
# It then loops through the incidents on the page and the uses parse_one_incident()
# function to grab the relevant incident info from the page.
#
# NOTE: The incidents are filtered to keep only those that occurred in the USA
# since our project is focused on Chicago, IL.
#
# Arguments:
#    a_html: A string of HTML content containing multiple incidents
#
# Returns:
#    retval: A list of dictionaries of USA incident information

def parse_incident_page(a_html):
    
    # Do a basic check
    if len(a_html) == 0:
        return None

    # Create BeautifulSoup object; parse with 'lxml'
    soup = BeautifulSoup(a_html, 'lxml')

    # Examine the results, then determine element that contains sought info
    # results are returned as an iterable list
    results = soup.find_all('div', class_='row div-report-box')

    # Keep track of how many entries we've added
    n_incidents = 0

    # Get info for all of the incidents on this page
    incident_list = []
    try:
        for r in results:

            # Parse this incident
            incident_info = parse_one_incident(r)
            #pprint(incident_info)

            # Only retain incidents in the United States
            # (Our scope is City of Chicago, so keeping all of USA should be sufficient)
            if "United States" in incident_info['incident_address_info']['incident_address']:
                # Append this USA incident to the list
                incident_list.append( incident_info )
                n_incidents += 1

                # Print a progress message
                # print(f">> Added incident #{n_incidents}: {incident_info['incident_title']}")

            #DEBUG ****************************************
            #if n_incidents > 3:
            #    break

    except TypeError:
        # If an iterable is not provided in "results", then fail gracefully
        pass
            
            
    # Return the list of dictionaries with USA incident info
    return incident_list

In [9]:
# URL of page to be scraped
# url_iwp = 'https://iwaspoisoned.com'
url_iwp = 'https://iwaspoisoned.com/?page=309'

# Visit the IWP page
browser.visit( url_iwp )

# Extract incidents from multiple pages
page_target = 20000

# How long to wait between pages to avoid triggering issues on website
page_wait = 2

# Count the number of pages visited
n_pages = 0

# Loop until no more pages or until page target is reached
full_incident_list = []
for j in range(page_target):
    # Get a page full of incidents from the USA
    i_list = parse_incident_page(browser.html)
    n_pages += 1
    
    # Add this list of incidents to a running list
    # full_incident_list.extend(i_list)
    
    # Add this list of incidents to the Mongo database
    # update_results = db.iwp.update_many({}, i_list, upsert=True)
    insert_results = db.iwp.insert_many(i_list)
        
    # Print a progress marker
    try:
        print(f"Page {n_pages}: {len(insert_results.inserted_ids)} of {len(i_list)} incidents added to DB. Total incidents: {db.iwp.count_documents({})}")

    except TypeError:
        print(f">> Page {n_pages}: TypeError")
    
    # Check to see if a hyperlink with attribute 'rel' = 'next' is present
    soup_thispage = BeautifulSoup(browser.html, 'lxml')
    next_tag = soup_thispage.find('a', {'rel' : 'next'})
        
    if next_tag:
        # Ok, there is a next page - get the hyperlink
        try:
            next_page_url = next_tag['href']
    
            # Wait for a specified number of seconds
            time.sleep(page_wait)

            # Click it!
            browser.click_link_by_href(next_page_url)

            #DEBUG ****************************************
            # if n_pages > 3:
            #    break
            
        # If KeyError occurs, then this tag has no html link for some reason
        except KeyError:
            break
            
    else:
        # No more pages - break out of this loop
        break


Page 1: 10 of 10 incidents added to DB. Total incidents: 2546
Page 2: 9 of 9 incidents added to DB. Total incidents: 2555
Page 3: 10 of 10 incidents added to DB. Total incidents: 2565
Page 4: 6 of 6 incidents added to DB. Total incidents: 2571
Page 5: 9 of 9 incidents added to DB. Total incidents: 2580
Page 6: 1 of 1 incidents added to DB. Total incidents: 2581
Page 7: 8 of 8 incidents added to DB. Total incidents: 2589
Page 8: 10 of 10 incidents added to DB. Total incidents: 2599
Page 9: 8 of 8 incidents added to DB. Total incidents: 2607
Page 10: 9 of 9 incidents added to DB. Total incidents: 2616
Page 11: 7 of 7 incidents added to DB. Total incidents: 2623
Page 12: 8 of 8 incidents added to DB. Total incidents: 2631
Page 13: 9 of 9 incidents added to DB. Total incidents: 2640
Page 14: 9 of 9 incidents added to DB. Total incidents: 2649
Page 15: 10 of 10 incidents added to DB. Total incidents: 2659
Page 16: 8 of 8 incidents added to DB. Total incidents: 2667
Page 17: 8 of 8 incidents

Page 134: 7 of 7 incidents added to DB. Total incidents: 3674
Page 135: 9 of 9 incidents added to DB. Total incidents: 3683
Page 136: 7 of 7 incidents added to DB. Total incidents: 3690
Page 137: 10 of 10 incidents added to DB. Total incidents: 3700
Page 138: 6 of 6 incidents added to DB. Total incidents: 3706
Page 139: 8 of 8 incidents added to DB. Total incidents: 3714
Page 140: 8 of 8 incidents added to DB. Total incidents: 3722
Page 141: 6 of 6 incidents added to DB. Total incidents: 3728
Page 142: 10 of 10 incidents added to DB. Total incidents: 3738
Page 143: 9 of 9 incidents added to DB. Total incidents: 3747
Page 144: 8 of 8 incidents added to DB. Total incidents: 3755
Page 145: 10 of 10 incidents added to DB. Total incidents: 3765
Page 146: 9 of 9 incidents added to DB. Total incidents: 3774
Page 147: 9 of 9 incidents added to DB. Total incidents: 3783
Page 148: 7 of 7 incidents added to DB. Total incidents: 3790
Page 149: 8 of 8 incidents added to DB. Total incidents: 3798
Pa

Page 266: 8 of 8 incidents added to DB. Total incidents: 4813
Page 267: 8 of 8 incidents added to DB. Total incidents: 4821
Page 268: 8 of 8 incidents added to DB. Total incidents: 4829
Page 269: 9 of 9 incidents added to DB. Total incidents: 4838
Page 270: 7 of 7 incidents added to DB. Total incidents: 4845
Page 271: 9 of 9 incidents added to DB. Total incidents: 4854
Page 272: 9 of 9 incidents added to DB. Total incidents: 4863
Page 273: 7 of 7 incidents added to DB. Total incidents: 4870
Page 274: 9 of 9 incidents added to DB. Total incidents: 4879
Page 275: 7 of 7 incidents added to DB. Total incidents: 4886
Page 276: 10 of 10 incidents added to DB. Total incidents: 4896
Page 277: 9 of 9 incidents added to DB. Total incidents: 4905
Page 278: 10 of 10 incidents added to DB. Total incidents: 4915
Page 279: 10 of 10 incidents added to DB. Total incidents: 4925
Page 280: 8 of 8 incidents added to DB. Total incidents: 4933
Page 281: 10 of 10 incidents added to DB. Total incidents: 4943


Page 398: 9 of 9 incidents added to DB. Total incidents: 5926
Page 399: 9 of 9 incidents added to DB. Total incidents: 5935
Page 400: 9 of 9 incidents added to DB. Total incidents: 5944
Page 401: 8 of 8 incidents added to DB. Total incidents: 5952
Page 402: 5 of 5 incidents added to DB. Total incidents: 5957
Page 403: 9 of 9 incidents added to DB. Total incidents: 5966
Page 404: 7 of 7 incidents added to DB. Total incidents: 5973
Page 405: 9 of 9 incidents added to DB. Total incidents: 5982
Page 406: 10 of 10 incidents added to DB. Total incidents: 5992
Page 407: 8 of 8 incidents added to DB. Total incidents: 6000
Page 408: 10 of 10 incidents added to DB. Total incidents: 6010
Page 409: 9 of 9 incidents added to DB. Total incidents: 6019
Page 410: 9 of 9 incidents added to DB. Total incidents: 6028
Page 411: 8 of 8 incidents added to DB. Total incidents: 6036
Page 412: 9 of 9 incidents added to DB. Total incidents: 6045
Page 413: 9 of 9 incidents added to DB. Total incidents: 6054
Page

Page 529: 8 of 8 incidents added to DB. Total incidents: 7066
Page 530: 10 of 10 incidents added to DB. Total incidents: 7076
Page 531: 8 of 8 incidents added to DB. Total incidents: 7084
Page 532: 9 of 9 incidents added to DB. Total incidents: 7093
Page 533: 8 of 8 incidents added to DB. Total incidents: 7101
Page 534: 7 of 7 incidents added to DB. Total incidents: 7108
Page 535: 6 of 6 incidents added to DB. Total incidents: 7114
Page 536: 9 of 9 incidents added to DB. Total incidents: 7123
Page 537: 10 of 10 incidents added to DB. Total incidents: 7133
Page 538: 10 of 10 incidents added to DB. Total incidents: 7143
Page 539: 8 of 8 incidents added to DB. Total incidents: 7151
Page 540: 8 of 8 incidents added to DB. Total incidents: 7159
Page 541: 10 of 10 incidents added to DB. Total incidents: 7169
Page 542: 9 of 9 incidents added to DB. Total incidents: 7178
Page 543: 7 of 7 incidents added to DB. Total incidents: 7185
Page 544: 7 of 7 incidents added to DB. Total incidents: 7192


...

Page 5753: 8 of 8 incidents added to DB. Total incidents: 53743
Page 5754: 10 of 10 incidents added to DB. Total incidents: 53753
Page 5755: 10 of 10 incidents added to DB. Total incidents: 53763
Page 5756: 10 of 10 incidents added to DB. Total incidents: 53773
Page 5757: 10 of 10 incidents added to DB. Total incidents: 53783
Page 5758: 9 of 9 incidents added to DB. Total incidents: 53792
Page 5759: 6 of 6 incidents added to DB. Total incidents: 53798
Page 5760: 9 of 9 incidents added to DB. Total incidents: 53807
Page 5761: 10 of 10 incidents added to DB. Total incidents: 53817
Page 5762: 9 of 9 incidents added to DB. Total incidents: 53826
Page 5763: 9 of 9 incidents added to DB. Total incidents: 53835
Page 5764: 9 of 9 incidents added to DB. Total incidents: 53844
Page 5765: 7 of 7 incidents added to DB. Total incidents: 53851
Page 5766: 8 of 8 incidents added to DB. Total incidents: 53859
Page 5767: 10 of 10 incidents added to DB. Total incidents: 53869
Page 5768: 9 of 9 incidents 

Page 5880: 10 of 10 incidents added to DB. Total incidents: 54899
Page 5881: 9 of 9 incidents added to DB. Total incidents: 54908
Page 5882: 10 of 10 incidents added to DB. Total incidents: 54918
Page 5883: 9 of 9 incidents added to DB. Total incidents: 54927
Page 5884: 7 of 7 incidents added to DB. Total incidents: 54934
Page 5885: 6 of 6 incidents added to DB. Total incidents: 54940
Page 5886: 8 of 8 incidents added to DB. Total incidents: 54948
Page 5887: 9 of 9 incidents added to DB. Total incidents: 54957
Page 5888: 9 of 9 incidents added to DB. Total incidents: 54966
Page 5889: 9 of 9 incidents added to DB. Total incidents: 54975
Page 5890: 9 of 9 incidents added to DB. Total incidents: 54984
Page 5891: 10 of 10 incidents added to DB. Total incidents: 54994
Page 5892: 9 of 9 incidents added to DB. Total incidents: 55003
Page 5893: 10 of 10 incidents added to DB. Total incidents: 55013
Page 5894: 7 of 7 incidents added to DB. Total incidents: 55020
Page 5895: 9 of 9 incidents adde

Page 6007: 8 of 8 incidents added to DB. Total incidents: 56059
Page 6008: 9 of 9 incidents added to DB. Total incidents: 56068
Page 6009: 8 of 8 incidents added to DB. Total incidents: 56076
Page 6010: 9 of 9 incidents added to DB. Total incidents: 56085
Page 6011: 9 of 9 incidents added to DB. Total incidents: 56094
Page 6012: 10 of 10 incidents added to DB. Total incidents: 56104
Page 6013: 8 of 8 incidents added to DB. Total incidents: 56112
Page 6014: 9 of 9 incidents added to DB. Total incidents: 56121
Page 6015: 9 of 9 incidents added to DB. Total incidents: 56130
Page 6016: 9 of 9 incidents added to DB. Total incidents: 56139
Page 6017: 9 of 9 incidents added to DB. Total incidents: 56148
Page 6018: 8 of 8 incidents added to DB. Total incidents: 56156
Page 6019: 8 of 8 incidents added to DB. Total incidents: 56164
Page 6020: 9 of 9 incidents added to DB. Total incidents: 56173
Page 6021: 9 of 9 incidents added to DB. Total incidents: 56182
Page 6022: 9 of 9 incidents added to D

Page 6134: 10 of 10 incidents added to DB. Total incidents: 57196
Page 6135: 8 of 8 incidents added to DB. Total incidents: 57204
Page 6136: 10 of 10 incidents added to DB. Total incidents: 57214
Page 6137: 10 of 10 incidents added to DB. Total incidents: 57224
Page 6138: 10 of 10 incidents added to DB. Total incidents: 57234
Page 6139: 8 of 8 incidents added to DB. Total incidents: 57242
Page 6140: 7 of 7 incidents added to DB. Total incidents: 57249
Page 6141: 8 of 8 incidents added to DB. Total incidents: 57257
Page 6142: 10 of 10 incidents added to DB. Total incidents: 57267
Page 6143: 9 of 9 incidents added to DB. Total incidents: 57276
Page 6144: 9 of 9 incidents added to DB. Total incidents: 57285
Page 6145: 10 of 10 incidents added to DB. Total incidents: 57295
Page 6146: 10 of 10 incidents added to DB. Total incidents: 57305
Page 6147: 8 of 8 incidents added to DB. Total incidents: 57313
Page 6148: 9 of 9 incidents added to DB. Total incidents: 57322
Page 6149: 10 of 10 incide

KeyboardInterrupt: 

In [ ]:
# Display items in MongoDB collection
all_incidents = db.iwp.find()

j=0
for i in all_incidents:
    print(f"{j}: {i['incident_title']}")
    j += 1

In [ ]:
i

In [ ]:
# Export the database to a JSON file
#o_file_iwp = "Resources/mongodb_export_iwp"

# Get an iterable pointer to the documents in the iwp collection
#cursor = db.iwp.find()

In [ ]:
# Open the export file
#file = open(o_file_iwp, 'w', encoding='utf-8')

# It's a list, so add an opening bracket
#file.write('[')

# Write each individual iwp document to the file
#for document in cursor:
    # file.write(json.dumps(document))
    # file.write(',')
    
    
# Add closing bracket for the list
#file.write(']')

# Just to be tidy, let's close the file
#file.close()